<p><strong><font size="6">WalOUS project</font></strong></p>

<p><strong><font size="6">C_3_Postclassifications</font></strong></p>

Jupyter Notebook containing the preprocessing steps consisting of: 
- Perform automated correction of "garden parcels" and add columns for tracking postclassification rules
- Sudbivide the residential classes into different levels of density.
- Create a new table for dissemination of results to end users and add attributes with all individual levels all attribute 'walousmaj' as well as two attributes 'hilucslanduse' compatible with two scenarios of HILUCS INSPIRE. 

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

# Define working environment

**Import libraries**

In [ ]:
# Import libraries needed for setting parameters of operating system 
import os
import sys

In [ ]:
## Import Psycopg2 library (interection with postgres database)
import psycopg2

**Add folder with SCR provided belong to this notebook**

In [ ]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

**Setup environment variables**

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [ ]:
run ../SRC/config.py

In [ ]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [ ]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

**Other functions**

In [ ]:
# Import functions for processing time information
import time
from processing_time import start_processing, print_processing_time
# Import function that check and create folder
from mkdir import check_create_dir

**Custom functions: Psycopg2 and Postgresql functions**

In [ ]:
# Import function that display postgresql's table header
from display_header import display_header
# Import function to creation connection to Postgresql database 
from postgres_functions import create_pg_connexion
# Import function to create index
from postgres_functions import create_index

In [ ]:
# Import function to export postgis table as shapefile or geopackage
from postgres_export import ogr2ogr_export
# Import functions for creation of postgresql back-up and restoration
from dump_functions import dump_table, dump_db, restore_psql

In [ ]:
# Import function to add columns for postclassification rule tracking
from postgres_postclassification import add_column_postclass_rulenumber
# Import function for fixing error of classification on residential garden
from postgres_postclassification import postclassif_residentialgardens_1
from postgres_postclassification import postclassif_residentialgardens_2
# Import function to subdivide residential classes into several level of neighborhood density
from postgres_postclassification import subdivide_residential_density
# Import function to create table CUSW with all columns for diffusion of result to end-users
from postgres_postclassification import create_cusw_table
# Import function to create columns with walousmaj at each level 
from postgres_postclassification import create_walousmaj_levels
# Import function to create column 'hilucslanduse_1' corresponding to scenario 1 of INSPIRE HILUCS 
from postgres_postclassification import create_hilucs_landuse_1
# Import function to create column 'hilucslanduse_1' corresponding to scenario 1 of INSPIRE HILUCS 
from postgres_postclassification import create_hilucs_landuse_2

# Add column for tracking postclassification rules

In [ ]:
# Name of table with classification results
classif_table = 'classification_results'

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Add column for 
add_column_postclass_rulenumber(con, result_table_schema='results', result_table_name=classif_table)
# Close connexion to postgres database
con.close()

# Postclassification of "garden parcels"

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Fix error of classification on residential garden - step 1 
postclassif_residentialgardens_1(con, result_table_schema='results', result_table_name=classif_table,
                               postclassif_rule=1, colum_label="walousmaj")
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Fix error of classification on residential garden - step 2
postclassif_residentialgardens_2(con, result_table_schema='results', result_table_name=classif_table,
                               postclassif_rule=2, colum_label="walousmaj")
# Close connexion to postgres database
con.close()

## Display

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
whereconditions = []
whereconditions.append("postclas_rule IS NOT NULL")
df = display_header(con, 'results', classif_table, where=' AND '.join(whereconditions), row_num=15)
# Close connexion to postgres database
con.close()
# Display dataframe
df

**Update index on walousmaj**

In [ ]:
# Add index on 'walousmaj'
con = create_pg_connexion(config_parameters)
create_index(con, 'results', classif_table, 'walousmaj')
con.close()

## Subdivide residential classes with neighborhood density

Subdivision of residential classes 5_1 and 5_2, based on the modal category of population density in surrounding neighborhoods computed on the RNPP points data.

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update column with classification label to subdivide classes '5_1' and '5_2' 
# according to the population density of the surrounding neighborhood
subdivide_residential_density(con, result_table_schema='results', result_table_name=classif_table,
                            colum_label="walousmaj")
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
whereconditions = []
whereconditions.append("walousmaj IN ('5_1_D','1_1','2_1')")
df = display_header(con, 'results', classif_table, where=' AND '.join(whereconditions), row_num=15)
# Close connexion to postgres database
con.close()
# Display dataframe
df

**Backup of table with classification results**

In [ ]:
# Create a back-up of a specific table
dump_table(config_parameters['pg_host'],config_parameters['pg_dbname'],config_parameters['pg_user'],
           config_parameters['pg_password'], 'results', classif_table, data['backup_classif_table'])

## Create new table with only columns to be shared with end-users

In [ ]:
# Name of table with classification results
cusw_table = 'cusw2018'

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create table with all result to be difused to the end-users (all_hilucs, walousmaj, 
# hilucslanduse_1, hilucslanduse_2) 
create_cusw_table(con, schema='results', input_table_name=classif_table, cusw_table_name=cusw_table)
# Close connexion to postgres database
con.close()

**Create index**

In [ ]:
# Add index on specific columns
con = create_pg_connexion(config_parameters)
create_index(con, 'results', cusw_table, 'capakey')
create_index(con, 'results', cusw_table, 'walousmaj')
create_index(con, 'results', cusw_table, 'geom', is_geom=True)
con.close()

**Backup of table CUSW2018**

In [ ]:
# Create a back-up of a specific table
dump_table(config_parameters['pg_host'],config_parameters['pg_dbname'],config_parameters['pg_user'],
           config_parameters['pg_password'], 'results', cusw_table, data['backup_cusw_table'])

## Create columns for all levels of "walousmaj"

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create columns for each level of the legend 
create_walousmaj_levels(con, result_table_schema='results', result_table_name=cusw_table,
                      colum_label="walousmaj")
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
whereconditions = []
whereconditions.append("walousmaj_l4 is not null")
df = display_header(con, 'results', cusw_table, where=' AND '.join(whereconditions), row_num=20)
# Close connexion to postgres database
con.close()
# Display dataframe
df

**Backup of table CUSW2018**

In [ ]:
# Create a back-up of a specific table
dump_table(config_parameters['pg_host'],config_parameters['pg_dbname'],config_parameters['pg_user'],
           config_parameters['pg_password'], 'results', cusw_table, data['backup_cusw_table'])

**Restore table CUSW2018**

## Create hilucslanduse column

A new array nammed hilucslanduse is created here. It is the INSPIRE compliant terminology of the land use classification. It consists in the concatenation of walousmaj and all_hilucs array elements. The value of walousmaj always remains at the fist position in the hilucslanduse array. 

In addition:
- Duplicates are removed. 
- In case an label of high level is redundant with a more precise label (of a lower level), the higher level is removed (e.g., if both '5_1' and '5' are present, only '5_1' is be kept). 
- For classes specific to walous projet and not compliant with HILUCS legend scheme, the label should be replace by the higher level, e.g., if '1_1_1_A' is present, it is replaced by 1_1_1.

In [ ]:
# Define a list of classes from 'walousmaj' or 'all_hilucs' that should be ignored when creating 'HilucsLandUse'
cl_ignore = ['8_8']

# Define a list of classes from 'walousmaj' that are not compliant 
# with INSPIRE and should be truncated of one level (e.g., '5_1_A' should become '5_1')
cl_truncate = ['1_1_1_A','1_1_1_B','1_1_1_C',
               '5_1_A','5_1_B','5_1_C','5_1_D',
               '5_2_A','5_2_B','5_2_C','5_2_D',
               '6_6_A','6_6_B']

# Define a lookup table (list of tupples) for convertion of nature conservation labels from walousmaj 
# to INSPIRE HILUCS legend
inspire_lookup = [('7','6_3'),('7_1','6_3_1'),('7_2','6_3_2')]

# Define a list of Hilucs classes that should be removed if presence of sublevels
# classes (more precise) classes in the array. E.g., is an array contains '4_1' and '4_1_1', the remaining 
# information will be only the most detailed classe which is '4_1_1'
cl_remove = ['1','2','3','4','5','6','1_1','3_3','3_4','4_1','4_3','6_3']

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create column HilucsLanduse compliant with INSPIRE
create_hilucs_landuse_1(con, result_table_schema='results', result_table_name=cusw_table,
                        cl_truncate=cl_truncate, cl_lookup=inspire_lookup,
                        colum_label="hilucslanduse_1")
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create column HilucsLanduse compliant with INSPIRE
create_hilucs_landuse_2(con, result_table_schema='results', result_table_name=cusw_table, 
                        cl_ignore=cl_ignore, cl_truncate=cl_truncate, 
                        cl_lookup=inspire_lookup ,cl_remove=cl_remove, colum_label="hilucslanduse_2")
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on specific columns
con = create_pg_connexion(config_parameters)
create_index(con, 'results', cusw_table, 'hilucslanduse_1')
create_index(con, 'results', cusw_table, 'hilucslanduse_2')
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
whereconditions = []
whereconditions.append("Cardinality(hilucslanduse_2) >= 2")
df = display_header(con, 'results', cusw_table, where=' AND '.join(whereconditions), row_num=10)
# Close connexion to postgres database
con.close()
# Display dataframe
df

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
whereconditions = []
#whereconditions.append("walousmaj IN ('7','7_1','7_2')")
whereconditions.append("walousmaj is null")
#whereconditions.append("capakey IN ('85040B0191/00A000','85040B0167/00_000')") #Parcels on which we debated during last SC about natural protected areas

df = display_header(con, 'results', cusw_table, where=' AND '.join(whereconditions), row_num=10)
# Close connexion to postgres database
con.close()
# Display dataframe
df

**Backup of table CUSW2018**

In [ ]:
# Create a back-up of a specific table
dump_table(config_parameters['pg_host'],config_parameters['pg_dbname'],config_parameters['pg_user'],
           config_parameters['pg_password'], 'results', cusw_table, data['backup_cusw_table'])

**Backup of the whole database**

In [ ]:
# Create a back-up of the whole database
dump_db(config_parameters['pg_host'],config_parameters['pg_dbname'],config_parameters['pg_user'],
           config_parameters['pg_password'], data['backup_db'])

# Export from PostgreSQL

## Export table Cusw2018 in EPSG:31370 (Belge 1972 / Belgian Lambert 72)

In [ ]:
# Export the table as Shapefile
query_export = "SELECT geom, capakey, \
lc_mode, lc_prop_2, lc_prop_3, lc_prop_5, lc_prop_6, lc_prop_11, lc_prop_12, lc_prop_41, lc_prop_42, \
rulebased_leaf, postclas_rule, \
walousmaj, walousmaj_l1, walousmaj_l2, walousmaj_l3, walousmaj_l4,\
hilucslanduse_1, array_to_string(hilucslanduse_2,',') as hilucslanduse_2 FROM results.%s"%cusw_table
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/classif_lu/%s_31370.shp"%cusw_table
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)

In [ ]:
# Export the table as Geopackage
query_export = "SELECT geom, capakey, \
lc_mode, lc_prop_2, lc_prop_3, lc_prop_5, lc_prop_6, lc_prop_11, lc_prop_12, lc_prop_41, lc_prop_42, \
rulebased_leaf, postclas_rule, \
walousmaj, walousmaj_l1, walousmaj_l2, walousmaj_l3, walousmaj_l4,\
hilucslanduse_1, array_to_string(hilucslanduse_2,',') as hilucslanduse_2 FROM results.%s"%cusw_table
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/classif_lu/%s_31370.gpkg"%cusw_table
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)

## Export table Cusw2018 in EPSG:3812 (ETRS89 / Belgian Lambert 2008)

In [ ]:
# Export the table as Shapefile
query_export = "SELECT ST_Transform(geom, 3812) as geom, capakey, \
lc_mode, lc_prop_2, lc_prop_3, lc_prop_5, lc_prop_6, lc_prop_11, lc_prop_12, lc_prop_41, lc_prop_42, \
rulebased_leaf, postclas_rule, \
walousmaj, walousmaj_l1, walousmaj_l2, walousmaj_l3, walousmaj_l4,\
hilucslanduse_1, array_to_string(hilucslanduse_2,',') as hilucslanduse_2 FROM results.%s"%cusw_table
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/classif_lu/%s_3812.shp"%cusw_table
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)

In [ ]:
# Export the table as Geopackage
query_export = "SELECT ST_Transform(geom, 3812) as geom, capakey, \
lc_mode,lc_prop_1,lc_prop_2,lc_prop_3,lc_prop_4,lc_prop_5,lc_prop_6,lc_prop_7,lc_prop_8,lc_prop_9,lc_prop_80,lc_prop_90, \
rulebased_leaf, postclas_rule, \
walousmaj, walousmaj_l1, walousmaj_l2, walousmaj_l3, walousmaj_l4,\
hilucslanduse_1, array_to_string(hilucslanduse_2,',') as hilucslanduse_2 FROM results.%s"%cusw_table
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/classif_lu/%s_3812.gpkg"%cusw_table
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)